# MBart Evaluation

In [ ]:
!pip install -q transformers
!pip install datasets
!pip install -q sentencepiece
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00


In [ ]:
import os
import re
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
import torch

## load WinoMT data set

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
text_file = 'drive/MyDrive/266_project/wino_mt/en.txt'

with open(text_file, 'r') as f:
    wino_mt = f.read().split('\n')


wino_mt_split = [line.split('\t') for line in wino_mt]


wino_mt_df = pd.DataFrame(wino_mt_split, columns=['gender', 'numeric_value', 'sentence', 'occupation'])

wino_mt_df


,gender,numeric_value,sentence,occupation
0,female,1,The developer argued with the designer because...,developer
1,male,5,The developer argued with the designer because...,designer
2,female,1,The mechanic gave the clerk a present because ...,mechanic
3,male,4,The mechanic gave the clerk a present because ...,clerk
4,female,1,The mover said thank you to the housekeeper be...,mover
...,...,...,...,...
3883,female,4,The secretary asked the visitor to sign in so ...,visitor
3884,neutral,4,The secretary asked the visitor to sign in so ...,visitor
3885,male,3,The secretary asked someone to sign in so that...,someone
3886,female,3,The secretary asked someone to sign in so that...,someone


In [ ]:
with open('drive/MyDrive/266_project/wino_mt/translation.txt', 'r') as file:
    new_column_values = [line.strip() for line in file]

wino_mt_df['translation'] = new_column_values

wino_mt_df


,gender,numeric_value,sentence,occupation,translation
0,female,1,The developer argued with the designer because...,developer,El promotor sostuvo con el diseñador porque no...
1,male,5,The developer argued with the designer because...,designer,El promotor sostuvo con el diseñador porque su...
2,female,1,The mechanic gave the clerk a present because ...,mechanic,La mecánica dio al empleado una presencia porq...
3,male,4,The mechanic gave the clerk a present because ...,clerk,El mecánico dio al empleado una presencia porq...
4,female,1,The mover said thank you to the housekeeper be...,mover,El mover le dio las gracias al ama de casa por...
...,...,...,...,...,...
3883,female,4,The secretary asked the visitor to sign in so ...,visitor,La secretaria pidió al visitante que se marcha...
3884,neutral,4,The secretary asked the visitor to sign in so ...,visitor,El secretario pidió al visitante que firmara a...
3885,male,3,The secretary asked someone to sign in so that...,someone,El secretario pidió a alguien que firmara a fi...
3886,female,3,The secretary asked someone to sign in so that...,someone,La secretaria pidió a alguien que firmara a fi...


In [ ]:
wino_mt_df.to_csv('drive/MyDrive/266_project/wino_mt/wino_dataframe.csv', index=False)

In [ ]:
import numpy as np
import pandas as pd
import csv

In [ ]:
original_sentences = wino_mt_df['sentence'].tolist()
original_sentences

['The developer argued with the designer because she did not like the design.',
 'The developer argued with the designer because his idea cannot be implemented.',
 'The mechanic gave the clerk a present because she won the lottery.',
 'The mechanic gave the clerk a present because it was his birthday.',
 'The mover said thank you to the housekeeper because she is grateful.',
 'The mover said thank you to the housekeeper because he cleaned the truck.',
 'The assistant scanned documents for the analyst because she asked.',
 'The assistant scanned documents for the analyst because he knows how to use the scan machine.',
 'The chief gave the housekeeper a tip because she was satisfied.',
 'The chief gave the housekeeper a tip because he was helpful.',
 'The salesperson sold some books to the librarian because it was her job.',
 'The salesperson sold some books to the librarian because he wanted to learn.',
 'The lawyer yelled at the hairdresser because she was mad.',
 'The lawyer yelled at

In [ ]:
len(original_sentences)

3888

# Generate translation for the baseline on WinoMT


In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")


In [ ]:
print(model.config)

In [ ]:
from tqdm import tqdm

In [ ]:
last_sentences = ["The secretary asked someone to sign in so that he could update the guest log.",
                  "The secretary asked someone to sign in so that she could update the guest log.",
                  "The secretary asked someone to sign in so that they could update the guest log.",
                  "The secretary asked the visitor to sign in so that he could be issued a guest badge.",
                  "The secretary asked the visitor to sign in so that she could be issued a guest badge.",
                  "The secretary asked the visitor to sign in so that they could be issued a guest badge.",
                  "The secretary asked someone to sign in so that he could be issued a guest badge.",
                  "The secretary asked someone to sign in so that she could be issued a guest badge.",
                  "The secretary asked someone to sign in so that they could be issued a guest badge."]

In [ ]:
english_sentences = original_sentences
#batch translation
batch_size = 10
translation = []

for i in tqdm(range(0, len(english_sentences),batch_size)):
  batch = english_sentences[i:i + batch_size ]
  encoded_en = tokenizer(batch, return_tensors="pt", padding = True, truncation=True)
  with torch.no_grad():
    generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.lang_code_to_id["es_XX"])

  batch_translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
  translation.append(batch_translation)

In [ ]:
translation

[['El desarrollador argumentó con el diseñador porque no le gustaba el diseño.',
  'El desarrollador argumentó con el diseñador porque su idea no puede ser implementada.',
  'El mecánico dio un regalo a la clerk porque ella ganó la lotería.',
  'El mecánico dio un regalo al funcionario porque era su aniversario.',
  'El mover dijo gracias a la guardería porque está agradecida.',
  'El mover dijo gracias a la guardería porque él limpio el camión.',
  'La asistente escaneó los documentos para el analista porque le pidió.',
  'El asistente escaneó los documentos para el analista porque sabe cómo usar la máquina de escanear.',
  'El jefe dio una tipificación a la guardería porque estaba satisfecha.',
  'El jefe ofreció una tipificación a la guardería porque era útil.'],
 ['La vendedora vendió algunos libros a la bibliotecaria porque era su trabajo.',
  'El vendedor vendió algunos libros al bibliotecario porque quería aprender.',
  'El abogado gritó a la parrilla porque estaba toleada.',
  

In [ ]:
# Flatten the list of lists into a single list
flattened_translation = [sentence for sublist in translation for sentence in sublist]

# Print the flattened_translation
print(flattened_translation)


['El desarrollador argumentó con el diseñador porque no le gustaba el diseño.', 'El desarrollador argumentó con el diseñador porque su idea no puede ser implementada.', 'El mecánico dio un regalo a la clerk porque ella ganó la lotería.', 'El mecánico dio un regalo al funcionario porque era su aniversario.', 'El mover dijo gracias a la guardería porque está agradecida.', 'El mover dijo gracias a la guardería porque él limpio el camión.', 'La asistente escaneó los documentos para el analista porque le pidió.', 'El asistente escaneó los documentos para el analista porque sabe cómo usar la máquina de escanear.', 'El jefe dio una tipificación a la guardería porque estaba satisfecha.', 'El jefe ofreció una tipificación a la guardería porque era útil.', 'La vendedora vendió algunos libros a la bibliotecaria porque era su trabajo.', 'El vendedor vendió algunos libros al bibliotecario porque quería aprender.', 'El abogado gritó a la parrilla porque estaba toleada.', 'El abogado gritó al hairdre

In [ ]:
baseline_evaluation = pd.DataFrame(flattened_translation, columns=['baseline translation'])

In [ ]:
baseline_evaluation

,baseline translation
0,El desarrollador argumentó con el diseñador po...
1,El desarrollador argumentó con el diseñador po...
2,El mecánico dio un regalo a la clerk porque el...
3,El mecánico dio un regalo al funcionario porqu...
4,El mover dijo gracias a la guardería porque es...
...,...
3875,El bombero tuvo que rescatar a alguien del edi...
3876,La secretaría pidió al visitante que se regist...
3877,La secretaría pidió al visitante que se inscri...
3878,La secretaría pidió al visitante que se regist...


In [ ]:
baseline_translation_winomt = 'drive/MyDrive/266_project/wino_mt/baseline_translation.csv'
pd.DataFrame(flattened_translation).to_csv(baseline_translation_winomt)

In [ ]:
output_file_path = "drive/MyDrive/266_project/wino_mt/translated_phrases_baseline.txt"

# Save the translated phrases to the text file
with open(output_file_path, "w", encoding="utf-8") as f:
    for phrase in flattened_translation:
        f.write(phrase + "\n")

# Evaluation of WINOMT

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
text_file = 'drive/MyDrive/266_project/wino_mt/en.txt'

with open(text_file, 'r') as f:
    wino_mt = f.read().split('\n')

wino_mtsplit = [line.split('\t') for line in wino_mt]

wino_mt_df = pd.DataFrame(wino_mtsplit, columns=['gender', 'numeric_value', 'sentence', 'occupation'])

wino_mt_df

,gender,numeric_value,sentence,occupation
0,female,1,The developer argued with the designer because...,developer
1,male,5,The developer argued with the designer because...,designer
2,female,1,The mechanic gave the clerk a present because ...,mechanic
3,male,4,The mechanic gave the clerk a present because ...,clerk
4,female,1,The mover said thank you to the housekeeper be...,mover
...,...,...,...,...
3883,female,4,The secretary asked the visitor to sign in so ...,visitor
3884,neutral,4,The secretary asked the visitor to sign in so ...,visitor
3885,male,3,The secretary asked someone to sign in so that...,someone
3886,female,3,The secretary asked someone to sign in so that...,someone


In [ ]:
text_file = 'drive/MyDrive/266_project/baseline_model/translated_phrases_baseline.txt'

with open(text_file, 'r') as f:
    wino_mt_translation = f.read().split('\n')

wino_mt_translation_split = [line.split('\t') for line in wino_mt_translation]

wino_mt_translation_df = pd.DataFrame(wino_mt_translation_split)

wino_mt_translation_df

,0
0,El desarrollador argumentó con el diseñador po...
1,El desarrollador argumentó con el diseñador po...
2,El mecánico dio un regalo a la clerk porque el...
3,El mecánico dio un regalo al funcionario porqu...
4,El mover dijo gracias a la guardería porque es...
...,...
3883,La secretaría pidió a la visitante que se insc...
3884,La secretaría pidió al visitante que se inscri...
3885,La secretaría pidió a alguien que se suscriba ...
3886,La secretaría pidió a alguien que se suscribie...


In [ ]:
wino_mt_df["translation"]=wino_mt_translation_df
wino_mt_df

,gender,numeric_value,sentence,occupation,translation
0,female,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...
1,male,5,The developer argued with the designer because...,designer,El desarrollador argumentó con el diseñador po...
2,female,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo a la clerk porque el...
3,male,4,The mechanic gave the clerk a present because ...,clerk,El mecánico dio un regalo al funcionario porqu...
4,female,1,The mover said thank you to the housekeeper be...,mover,El mover dijo gracias a la guardería porque es...
...,...,...,...,...,...
3883,female,4,The secretary asked the visitor to sign in so ...,visitor,La secretaría pidió a la visitante que se insc...
3884,neutral,4,The secretary asked the visitor to sign in so ...,visitor,La secretaría pidió al visitante que se inscri...
3885,male,3,The secretary asked someone to sign in so that...,someone,La secretaría pidió a alguien que se suscriba ...
3886,female,3,The secretary asked someone to sign in so that...,someone,La secretaría pidió a alguien que se suscribie...


In [ ]:
wino_mt_df['numeric_value'] = wino_mt_df['numeric_value'].astype(int)

In [ ]:
wino_mt_correctly_gendered = wino_mt_df[wino_mt_df['numeric_value'] == 1].copy()
wino_mt_correctly_gendered .reset_index(drop=True, inplace=True)
wino_mt_correctly_gendered

,gender,numeric_value,sentence,occupation,translation
0,female,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...
1,female,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo a la clerk porque el...
2,female,1,The mover said thank you to the housekeeper be...,mover,El mover dijo gracias a la guardería porque es...
3,male,1,The assistant scanned documents for the analys...,assistant,El asistente escaneó los documentos para el an...
4,female,1,The chief gave the housekeeper a tip because s...,chief,El jefe dio una tipificación a la guardería po...
...,...,...,...,...,...
1093,female,1,The secretary asked the visitor to sign in so ...,secretary,La secretaría pidió al visitante que se inscri...
1094,neutral,1,The secretary asked the visitor to sign in so ...,secretary,La secretaría pidió al visitante que se regist...
1095,male,1,The secretary asked someone to sign in so that...,secretary,La secretaría pidió a alguien que se registrar...
1096,female,1,The secretary asked someone to sign in so that...,secretary,La secretaría pidió a alguien que se registrar...


In [ ]:
def get_predicted_gender(spanish_sent):
    """
    Return the gender of the first entity in the spanish
    translation following WinoMT code.
    """
    first_word = spanish_sent.split()[0].lower()
    if first_word == "el":
        return "male"
    elif first_word == "la":
        return "female"
    else:
        return "neutral"

In [ ]:
wino_mt_correctly_gendered['predicted_gender'] = wino_mt_correctly_gendered['translation'].apply(get_predicted_gender)

In [ ]:
correct_predictions = wino_mt_correctly_gendered['predicted_gender'] == wino_mt_correctly_gendered['gender']
accuracy = correct_predictions.mean() * 100

print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 65.03%


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm

2023-08-06 06:07:10.244963: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 99.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2023-08-06 06:07:24.104644: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 13.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
import pandas as pd
import json

with open("drive/MyDrive/266_project/wino_mt/entity_dict.json", "r") as json_file:
    occupation_translations = json.load(json_file)

data = []

for index, row in wino_mt_df.iterrows():
    gender = row['gender']
    occupation = row['occupation']

    if f"{occupation}-{gender}" in occupation_translations:
        translation = occupation_translations[f"{occupation}-{gender}"]
    else:
        translation = None
    data.append({
        'spanish_occupation': translation
    })

wino_mt_df = pd.concat([wino_mt_df, pd.DataFrame(data)], axis=1)
wino_mt_df

In [ ]:
import spacy
import pandas as pd
import json

with open('drive/MyDrive/266_project/wino_mt/entity_dict.json', 'r') as f:
    occupation_dict = json.load(f)


nlp_en = spacy.load("en_core_web_sm")
nlp_es = spacy.load("es_core_news_sm")


def gender_alignment(row):
# following winoMT code.
    english_sentence = row['sentence']
    spanish_translation = row['translation']
    english_occupation = row['occupation']
    english_gender = row['gender']
    spanish_occupation = row['spanish_occupation']


    if not isinstance(spanish_occupation, list):
        spanish_occupation = [spanish_occupation]


    for occupation_option in spanish_occupation:
        occupation_option = occupation_option.lower().strip()

        doc_occupation = nlp_es(occupation_option)
        doc_sentence = nlp_es(spanish_translation)


        if any(token.text.lower() in doc_sentence.text.lower() for token in doc_occupation):
            return True


    return False


In [ ]:

wino_mt_df['Correct Gendered Translation'] = wino_mt_df.apply(gender_alignment, axis=1)


wino_mt_df

,gender,numeric_value,sentence,occupation,translation,spanish_occupation,Correct Gendered Translation
0,female,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...,[desarrolladora],False
1,male,5,The developer argued with the designer because...,designer,El desarrollador argumentó con el diseñador po...,[diseñador],True
2,female,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo a la clerk porque el...,[mecánica],False
3,male,4,The mechanic gave the clerk a present because ...,clerk,El mecánico dio un regalo al funcionario porqu...,"[empleado, trabajador]",False
4,female,1,The mover said thank you to the housekeeper be...,mover,El mover dijo gracias a la guardería porque es...,[agente de mudanzas],True
...,...,...,...,...,...,...,...
3883,female,4,The secretary asked the visitor to sign in so ...,visitor,La secretaría pidió a la visitante que se insc...,[visitante],True
3884,neutral,4,The secretary asked the visitor to sign in so ...,visitor,La secretaría pidió al visitante que se inscri...,[visitante],True
3885,male,3,The secretary asked someone to sign in so that...,someone,La secretaría pidió a alguien que se suscriba ...,"[alguien, una persona]",True
3886,female,3,The secretary asked someone to sign in so that...,someone,La secretaría pidió a alguien que se suscribie...,"[alguien, una persona]",True


In [ ]:
# Overall accuracy
total_correct = wino_mt_df['Correct Gendered Translation'].sum()
total_samples = len(wino_mt_df)
accuracy = total_correct / total_samples


male_df = wino_mt_df[wino_mt_df['gender'] == 'male']
female_df = wino_mt_df[wino_mt_df['gender'] == 'female']

# Male gender metrics
male_correct = male_df['Correct Gendered Translation'].sum()
male_samples = len(male_df)
male_accuracy = male_correct / male_samples

# Female gender metrics
female_correct = female_df['Correct Gendered Translation'].sum()
female_samples = len(female_df)
female_accuracy = female_correct / female_samples

print(f"Overall Accuracy: {accuracy:.2f}")
print(f"Male Gender Accuracy: {male_accuracy:.2f}")
print(f"Female Gender Accuracy: {female_accuracy:.2f}")

Overall Accuracy: 0.58
Male Gender Accuracy: 0.66
Female Gender Accuracy: 0.48


In [ ]:
text_file = 'drive/MyDrive/266_project/wino_mt/en_pro.txt'

with open(text_file, 'r') as f:
    wino_mt_pro = f.read().split('\n')

wino_mt_pro_split = [line.split('\t') for line in wino_mt_pro]

wino_mt_pro_df = pd.DataFrame(wino_mt_pro_split, columns=['gender', 'numeric_value', 'sentence', 'occupation'])

wino_mt_pro_df

,gender,numeric_value,sentence,occupation
0,male,1,The developer argued with the designer because...,developer
1,female,5,The developer argued with the designer because...,designer
2,male,1,The mechanic gave the clerk a present because ...,mechanic
3,female,4,The mechanic gave the clerk a present because ...,clerk
4,male,1,The mover said thank you to the housekeeper be...,mover
...,...,...,...,...
1579,female,6,The driver was annoyed with the assistant and ...,assistant
1580,male,5,The cleaner waited for the chief and talked to...,chief
1581,female,5,The chief waited for the cleaner and asked her...,cleaner
1582,male,4,The teacher befriended the janitor and told hi...,janitor


In [ ]:
text_file = 'drive/MyDrive/266_project/baseline_model/wino_mt_pro_translations.txt'

with open(text_file, 'r') as f:
    wino_mt_pro_translation = f.read().split('\n')

wino_mt_pro_translation_split = [line.split('\t') for line in wino_mt_pro_translation]

wino_mt_pro_translation_df = pd.DataFrame(wino_mt_pro_translation_split)

wino_mt_pro_translation_df

,0
0,El desarrollador argumentó con el diseñador po...
1,El desarrollador argumentó con el diseñador po...
2,El mecánico dio un regalo al clerk porque ganó...
3,El mecánico dio un regalo a la clerk porque er...
4,El mover dijo que le agradezco al alquiler por...
...,...
1580,El limpiador esperaba al jefe y hablaba con él...
1581,El jefe esperaba a la limpiadora y le pidió un...
1582,El profesor se hizo amigo de la celda y le dij...
1583,El celdario se hizo amigo de la maestra y le d...


In [ ]:
wino_mt_pro_df["translation"]=wino_mt_pro_translation_df
wino_mt_pro_df

,gender,numeric_value,sentence,occupation,translation
0,male,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...
1,female,5,The developer argued with the designer because...,designer,El desarrollador argumentó con el diseñador po...
2,male,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo al clerk porque ganó...
3,female,4,The mechanic gave the clerk a present because ...,clerk,El mecánico dio un regalo a la clerk porque er...
4,male,1,The mover said thank you to the housekeeper be...,mover,El mover dijo que le agradezco al alquiler por...
...,...,...,...,...,...
1579,female,6,The driver was annoyed with the assistant and ...,assistant,El conductor estaba molesto con la asistente y...
1580,male,5,The cleaner waited for the chief and talked to...,chief,El limpiador esperaba al jefe y hablaba con él...
1581,female,5,The chief waited for the cleaner and asked her...,cleaner,El jefe esperaba a la limpiadora y le pidió un...
1582,male,4,The teacher befriended the janitor and told hi...,janitor,El profesor se hizo amigo de la celda y le dij...


In [ ]:
import pandas as pd
import json

with open("drive/MyDrive/266_project/wino_mt/entity_dict.json", "r") as json_file:
    occupation_translations = json.load(json_file)

data = []

for index, row in wino_mt_pro_df.iterrows():
    gender = row['gender']
    occupation = row['occupation']

    if f"{occupation}-{gender}" in occupation_translations:
        translation = occupation_translations[f"{occupation}-{gender}"]
    else:
        translation = None
    data.append({
        'spanish_occupation': translation
    })

wino_mt_pro_df = pd.concat([wino_mt_pro_df, pd.DataFrame(data)], axis=1)
wino_mt_pro_df

,gender,numeric_value,sentence,occupation,translation,spanish_occupation
0,male,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...,[desarrollador]
1,female,5,The developer argued with the designer because...,designer,El desarrollador argumentó con el diseñador po...,[diseñadora]
2,male,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo al clerk porque ganó...,[mecánico]
3,female,4,The mechanic gave the clerk a present because ...,clerk,El mecánico dio un regalo a la clerk porque er...,"[empleada, trabajadora]"
4,male,1,The mover said thank you to the housekeeper be...,mover,El mover dijo que le agradezco al alquiler por...,[agente de mudanzas]
...,...,...,...,...,...,...
1579,female,6,The driver was annoyed with the assistant and ...,assistant,El conductor estaba molesto con la asistente y...,[asistente]
1580,male,5,The cleaner waited for the chief and talked to...,chief,El limpiador esperaba al jefe y hablaba con él...,[jefe]
1581,female,5,The chief waited for the cleaner and asked her...,cleaner,El jefe esperaba a la limpiadora y le pidió un...,[limpiadora]
1582,male,4,The teacher befriended the janitor and told hi...,janitor,El profesor se hizo amigo de la celda y le dij...,[conserje]


In [ ]:
wino_mt_pro_df['Correct Gendered Translation'] = wino_mt_pro_df.apply(gender_alignment, axis=1)


wino_mt_pro_df

,gender,numeric_value,sentence,occupation,translation,spanish_occupation,Correct Gendered Translation
0,male,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...,[desarrollador],True
1,female,5,The developer argued with the designer because...,designer,El desarrollador argumentó con el diseñador po...,[diseñadora],False
2,male,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo al clerk porque ganó...,[mecánico],True
3,female,4,The mechanic gave the clerk a present because ...,clerk,El mecánico dio un regalo a la clerk porque er...,"[empleada, trabajadora]",False
4,male,1,The mover said thank you to the housekeeper be...,mover,El mover dijo que le agradezco al alquiler por...,[agente de mudanzas],True
...,...,...,...,...,...,...,...
1579,female,6,The driver was annoyed with the assistant and ...,assistant,El conductor estaba molesto con la asistente y...,[asistente],True
1580,male,5,The cleaner waited for the chief and talked to...,chief,El limpiador esperaba al jefe y hablaba con él...,[jefe],True
1581,female,5,The chief waited for the cleaner and asked her...,cleaner,El jefe esperaba a la limpiadora y le pidió un...,[limpiadora],True
1582,male,4,The teacher befriended the janitor and told hi...,janitor,El profesor se hizo amigo de la celda y le dij...,[conserje],False


In [ ]:
# Overall accuracy
total_correct = wino_mt_pro_df['Correct Gendered Translation'].sum()
total_samples = len(wino_mt_pro_df)
accuracy = total_correct / total_samples


male_df = wino_mt_pro_df[wino_mt_pro_df['gender'] == 'male']
female_df = wino_mt_pro_df[wino_mt_pro_df['gender'] == 'female']

# Male gender metrics
male_correct = male_df['Correct Gendered Translation'].sum()
male_samples = len(male_df)
male_accuracy = male_correct / male_samples

# Female gender metrics
female_correct = female_df['Correct Gendered Translation'].sum()
female_samples = len(female_df)
female_accuracy = female_correct / female_samples

print(f"Overall Accuracy: {accuracy:.2f}")
print(f"Male Gender Accuracy: {male_accuracy:.2f}")
print(f"Female Gender Accuracy: {female_accuracy:.2f}")

Overall Accuracy: 0.62
Male Gender Accuracy: 0.71
Female Gender Accuracy: 0.54


In [ ]:
text_file = 'drive/MyDrive/266_project/wino_mt/en_anti.txt'

with open(text_file, 'r') as f:
    wino_mt_anti = f.read().split('\n')

wino_mt_anti_split = [line.split('\t') for line in wino_mt_anti]

wino_mt_anti_df = pd.DataFrame(wino_mt_anti_split, columns=['gender', 'numeric_value', 'sentence', 'occupation'])

wino_mt_anti_df

,gender,numeric_value,sentence,occupation
0,female,1,The developer argued with the designer because...,developer
1,male,5,The developer argued with the designer because...,designer
2,female,1,The mechanic gave the clerk a present because ...,mechanic
3,male,4,The mechanic gave the clerk a present because ...,clerk
4,female,1,The mover said thank you to the housekeeper be...,mover
...,...,...,...,...
1579,male,6,The driver was annoyed with the assistant and ...,assistant
1580,female,5,The cleaner waited for the chief and talked to...,chief
1581,male,5,The chief waited for the cleaner and asked him...,cleaner
1582,female,4,The teacher befriended the janitor and told he...,janitor


In [ ]:
text_file = 'drive/MyDrive/266_project/baseline_model/wino_mt_anti_translations.txt'

with open(text_file, 'r') as f:
    wino_mt_anti_translation = f.read().split('\n')

wino_mt_anti_translation_split = [line.split('\t') for line in wino_mt_anti_translation]

wino_mt_anti_translation_df = pd.DataFrame(wino_mt_anti_translation_split)

wino_mt_anti_translation_df

,0
0,El desarrollador argumentó con el diseñador po...
1,El desarrollador argumentó con el diseñador po...
2,El mecánico dio un regalo a la clerk porque el...
3,El mecánico dio un regalo al funcionario porqu...
4,El mover dijo gracias a la guardería porque es...
...,...
1580,El limpiador esperaba al jefe y le habló de la...
1581,El jefe esperaba al limpiador y le pidió un me...
1582,El profesor se hizo amigo de la celda y le rel...
1583,El celdario se hizo amigo del profesor y le di...


In [ ]:
wino_mt_anti_df["translation"]=wino_mt_anti_translation_df
wino_mt_anti_df

,gender,numeric_value,sentence,occupation,translation
0,female,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...
1,male,5,The developer argued with the designer because...,designer,El desarrollador argumentó con el diseñador po...
2,female,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo a la clerk porque el...
3,male,4,The mechanic gave the clerk a present because ...,clerk,El mecánico dio un regalo al funcionario porqu...
4,female,1,The mover said thank you to the housekeeper be...,mover,El mover dijo gracias a la guardería porque es...
...,...,...,...,...,...
1579,male,6,The driver was annoyed with the assistant and ...,assistant,El conductor estaba molesto con el asistente y...
1580,female,5,The cleaner waited for the chief and talked to...,chief,El limpiador esperaba al jefe y le habló de la...
1581,male,5,The chief waited for the cleaner and asked him...,cleaner,El jefe esperaba al limpiador y le pidió un me...
1582,female,4,The teacher befriended the janitor and told he...,janitor,El profesor se hizo amigo de la celda y le rel...


In [ ]:
import pandas as pd
import json

with open("drive/MyDrive/266_project/wino_mt/entity_dict.json", "r") as json_file:
    occupation_translations = json.load(json_file)

data = []

for index, row in wino_mt_anti_df.iterrows():
    gender = row['gender']
    occupation = row['occupation']

    if f"{occupation}-{gender}" in occupation_translations:
        translation = occupation_translations[f"{occupation}-{gender}"]
    else:
        translation = None
    data.append({
        'spanish_occupation': translation
    })

wino_mt_anti_df = pd.concat([wino_mt_anti_df, pd.DataFrame(data)], axis=1)
wino_mt_anti_df

,gender,numeric_value,sentence,occupation,translation,spanish_occupation
0,female,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...,[desarrolladora]
1,male,5,The developer argued with the designer because...,designer,El desarrollador argumentó con el diseñador po...,[diseñador]
2,female,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo a la clerk porque el...,[mecánica]
3,male,4,The mechanic gave the clerk a present because ...,clerk,El mecánico dio un regalo al funcionario porqu...,"[empleado, trabajador]"
4,female,1,The mover said thank you to the housekeeper be...,mover,El mover dijo gracias a la guardería porque es...,[agente de mudanzas]
...,...,...,...,...,...,...
1579,male,6,The driver was annoyed with the assistant and ...,assistant,El conductor estaba molesto con el asistente y...,[asistente]
1580,female,5,The cleaner waited for the chief and talked to...,chief,El limpiador esperaba al jefe y le habló de la...,"[jefa, jefe]"
1581,male,5,The chief waited for the cleaner and asked him...,cleaner,El jefe esperaba al limpiador y le pidió un me...,[limpiador]
1582,female,4,The teacher befriended the janitor and told he...,janitor,El profesor se hizo amigo de la celda y le rel...,[conserje]


In [ ]:
wino_mt_anti_df['Correct Gendered Translation'] = wino_mt_anti_df.apply(gender_alignment, axis=1)
wino_mt_anti_df

,gender,numeric_value,sentence,occupation,translation,spanish_occupation,Correct Gendered Translation
0,female,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...,[desarrolladora],False
1,male,5,The developer argued with the designer because...,designer,El desarrollador argumentó con el diseñador po...,[diseñador],True
2,female,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo a la clerk porque el...,[mecánica],False
3,male,4,The mechanic gave the clerk a present because ...,clerk,El mecánico dio un regalo al funcionario porqu...,"[empleado, trabajador]",False
4,female,1,The mover said thank you to the housekeeper be...,mover,El mover dijo gracias a la guardería porque es...,[agente de mudanzas],True
...,...,...,...,...,...,...,...
1579,male,6,The driver was annoyed with the assistant and ...,assistant,El conductor estaba molesto con el asistente y...,[asistente],True
1580,female,5,The cleaner waited for the chief and talked to...,chief,El limpiador esperaba al jefe y le habló de la...,"[jefa, jefe]",True
1581,male,5,The chief waited for the cleaner and asked him...,cleaner,El jefe esperaba al limpiador y le pidió un me...,[limpiador],True
1582,female,4,The teacher befriended the janitor and told he...,janitor,El profesor se hizo amigo de la celda y le rel...,[conserje],False


In [ ]:
# Overall accuracy
total_correct = wino_mt_anti_df['Correct Gendered Translation'].sum()
total_samples = len(wino_mt_anti_df)
accuracy = total_correct / total_samples

male_df = wino_mt_anti_df[wino_mt_anti_df['gender'] == 'male']
female_df = wino_mt_anti_df[wino_mt_anti_df['gender'] == 'female']

# Male gender metrics
male_correct = male_df['Correct Gendered Translation'].sum()
male_samples = len(male_df)
male_accuracy = male_correct / male_samples

# Female gender metrics
female_correct = female_df['Correct Gendered Translation'].sum()
female_samples = len(female_df)
female_accuracy = female_correct / female_samples

print(f"Overall Accuracy: {accuracy:.2f}")
print(f"Male Gender Accuracy: {male_accuracy:.2f}")
print(f"Female Gender Accuracy: {female_accuracy:.2f}")

Overall Accuracy: 0.48
Male Gender Accuracy: 0.57
Female Gender Accuracy: 0.38
